In [1]:
import random
from random import randint
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [17]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [81]:
EPISODES = 5000

In [88]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.99    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.995
        self.learning_rate = 0.01
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state,collision_flag):
        self.memory.append((state, action, reward, next_state,collision_flag))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
      
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, collision_flag in minibatch:
            target = reward
            if not collision_flag:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [89]:
state_size = 1
action_size = 6
agent = DQNAgent(state_size, action_size)


batch_size = 32



In [90]:
for e in range(1,EPISODES+1):
    num_env_cars = randint(1,5) + 20;  # random car number
    eng.env_ini(num_env_cars,nargout=0)
    
    time = 0;
    dist = 0;
    score = 0;
  
    eng.deep_get_Observation(1,nargout=0)
    collision_flag=False
    
    
    
    for time in range(1,201):
        
        state = eng.deep_obs2state(nargout=1)
    
        state = np.reshape(state, [1, state_size])
        
        eng.old_observation(nargout=0)
        
        action = agent.act(state)
        action=np.asscalar(np.int64(action))
            
        dist=eng.deepQ_update_dynamics(dist,action+1,nargout=1)
            
        collision_flag = eng.deep_check_collision(nargout=1)
            
        eng.deep_get_Observation(1,nargout=0)
            
        reward = eng.deep_get_reward(collision_flag,nargout=1)
            
           
        next_state = eng.deep_obs2state(nargout=1)
            
        next_state = np.reshape(next_state, [1, state_size])
            
            
        agent.remember(state, action, reward, next_state,collision_flag)
            
            
        
        if collision_flag:

            break
    
    score = (dist-4000)/1000;     
    print("episode: {}/{}, score: {}, e: {:.2}, Collision: {} "
              .format(e, EPISODES, score, agent.epsilon,collision_flag))
            
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
    if e % 10 == 0:
        agent.save("dqn.h5")
                
        

episode: 1/5000, score: -3.9445, e: 1.0, Collision: True 
episode: 2/5000, score: -3.5585, e: 1.0, Collision: True 
episode: 3/5000, score: -3.8675, e: 1.0, Collision: True 
episode: 4/5000, score: -3.97, e: 1.0, Collision: True 
episode: 5/5000, score: -3.8565, e: 1.0, Collision: True 
episode: 6/5000, score: -3.91, e: 0.99, Collision: True 
episode: 7/5000, score: -3.97, e: 0.99, Collision: True 
episode: 8/5000, score: -3.933, e: 0.99, Collision: True 
episode: 9/5000, score: -3.7575, e: 0.98, Collision: True 
episode: 10/5000, score: -3.889, e: 0.98, Collision: True 
episode: 11/5000, score: -2.922, e: 0.97, Collision: True 
episode: 12/5000, score: -3.973, e: 0.97, Collision: True 
episode: 13/5000, score: -3.4885, e: 0.96, Collision: True 
episode: 14/5000, score: -3.221, e: 0.96, Collision: True 
episode: 15/5000, score: -3.771, e: 0.95, Collision: True 
episode: 16/5000, score: -3.8405, e: 0.95, Collision: True 
episode: 17/5000, score: -3.107, e: 0.94, Collision: True 
episode

episode: 140/5000, score: -2.428, e: 0.51, Collision: True 
episode: 141/5000, score: -3.973, e: 0.51, Collision: True 
episode: 142/5000, score: -2.0695, e: 0.5, Collision: True 
episode: 143/5000, score: -3.4915, e: 0.5, Collision: True 
episode: 144/5000, score: -3.849, e: 0.5, Collision: True 
episode: 145/5000, score: -2.88, e: 0.5, Collision: True 
episode: 146/5000, score: -0.951, e: 0.49, Collision: True 
episode: 147/5000, score: -2.11, e: 0.49, Collision: True 
episode: 148/5000, score: -1.856, e: 0.49, Collision: True 
episode: 149/5000, score: -0.243, e: 0.49, Collision: True 
episode: 150/5000, score: -3.1115, e: 0.48, Collision: True 
episode: 151/5000, score: -3.7335, e: 0.48, Collision: True 
episode: 152/5000, score: -0.496, e: 0.48, Collision: True 
episode: 153/5000, score: -3.5445, e: 0.48, Collision: True 
episode: 154/5000, score: -3.863, e: 0.47, Collision: True 
episode: 155/5000, score: -3.954, e: 0.47, Collision: True 
episode: 156/5000, score: -3.8535, e: 0.4

episode: 277/5000, score: 0.084, e: 0.26, Collision: False 
episode: 278/5000, score: -3.88, e: 0.25, Collision: True 
episode: 279/5000, score: -1.8965, e: 0.25, Collision: True 
episode: 280/5000, score: -3.745, e: 0.25, Collision: True 
episode: 281/5000, score: -0.9055, e: 0.25, Collision: True 
episode: 282/5000, score: 0.0935, e: 0.25, Collision: False 
episode: 283/5000, score: -2.1225, e: 0.25, Collision: True 
episode: 284/5000, score: -3.946, e: 0.25, Collision: True 
episode: 285/5000, score: -3.848, e: 0.25, Collision: True 
episode: 286/5000, score: -3.8675, e: 0.24, Collision: True 
episode: 287/5000, score: -1.9145, e: 0.24, Collision: True 
episode: 288/5000, score: -2.975, e: 0.24, Collision: True 
episode: 289/5000, score: 0.052, e: 0.24, Collision: False 
episode: 290/5000, score: -2.7295, e: 0.24, Collision: True 
episode: 291/5000, score: -3.82, e: 0.24, Collision: True 
episode: 292/5000, score: -3.772, e: 0.24, Collision: True 
episode: 293/5000, score: -3.678, e

episode: 414/5000, score: -3.7935, e: 0.13, Collision: True 
episode: 415/5000, score: 0.0375, e: 0.13, Collision: False 
episode: 416/5000, score: 0.0315, e: 0.13, Collision: False 
episode: 417/5000, score: -2.4535, e: 0.13, Collision: True 
episode: 418/5000, score: 0.0365, e: 0.13, Collision: False 
episode: 419/5000, score: -2.0205, e: 0.13, Collision: True 
episode: 420/5000, score: -3.8545, e: 0.12, Collision: True 
episode: 421/5000, score: -2.012, e: 0.12, Collision: True 
episode: 422/5000, score: -3.939, e: 0.12, Collision: True 
episode: 423/5000, score: 0.0155, e: 0.12, Collision: False 
episode: 424/5000, score: -3.879, e: 0.12, Collision: True 
episode: 425/5000, score: -3.875, e: 0.12, Collision: True 
episode: 426/5000, score: 0.0245, e: 0.12, Collision: False 
episode: 427/5000, score: 0.027, e: 0.12, Collision: False 
episode: 428/5000, score: -2.127, e: 0.12, Collision: True 
episode: 429/5000, score: 0.0255, e: 0.12, Collision: False 
episode: 430/5000, score: -3.6

episode: 552/5000, score: -3.853, e: 0.1, Collision: True 
episode: 553/5000, score: -3.3725, e: 0.1, Collision: True 
episode: 554/5000, score: -3.82, e: 0.1, Collision: True 
episode: 555/5000, score: -3.9315, e: 0.1, Collision: True 
episode: 556/5000, score: 0.026, e: 0.1, Collision: False 
episode: 557/5000, score: -3.85, e: 0.1, Collision: True 
episode: 558/5000, score: -3.6745, e: 0.1, Collision: True 
episode: 559/5000, score: -3.704, e: 0.1, Collision: True 
episode: 560/5000, score: -2.556, e: 0.1, Collision: True 
episode: 561/5000, score: 0.017, e: 0.1, Collision: False 
episode: 562/5000, score: -3.497, e: 0.1, Collision: True 
episode: 563/5000, score: -2.342, e: 0.1, Collision: True 
episode: 564/5000, score: 0.0165, e: 0.1, Collision: False 
episode: 565/5000, score: 0.02, e: 0.1, Collision: False 
episode: 566/5000, score: 0.046, e: 0.1, Collision: False 
episode: 567/5000, score: -3.61, e: 0.1, Collision: True 
episode: 568/5000, score: -3.865, e: 0.1, Collision: Tru

...

episode: 4399/5000, score: -3.91, e: 0.1, Collision: True 
episode: 4400/5000, score: -3.6155, e: 0.1, Collision: True 
episode: 4401/5000, score: -3.4, e: 0.1, Collision: True 
episode: 4402/5000, score: -3.91, e: 0.1, Collision: True 
episode: 4403/5000, score: -3.94, e: 0.1, Collision: True 
episode: 4404/5000, score: -3.47, e: 0.1, Collision: True 
episode: 4405/5000, score: -2.601, e: 0.1, Collision: True 
episode: 4406/5000, score: -3.2595, e: 0.1, Collision: True 
episode: 4407/5000, score: -3.823, e: 0.1, Collision: True 
episode: 4408/5000, score: -3.824, e: 0.1, Collision: True 
episode: 4409/5000, score: -3.8, e: 0.1, Collision: True 
episode: 4410/5000, score: -3.85, e: 0.1, Collision: True 
episode: 4411/5000, score: -3.973, e: 0.1, Collision: True 
episode: 4412/5000, score: -3.48, e: 0.1, Collision: True 
episode: 4413/5000, score: -3.7, e: 0.1, Collision: True 
episode: 4414/5000, score: -3.94, e: 0.1, Collision: True 
episode: 4415/5000, score: -3.94, e: 0.1, Collision

episode: 4536/5000, score: 1.9765, e: 0.1, Collision: False 
episode: 4537/5000, score: -3.88, e: 0.1, Collision: True 
episode: 4538/5000, score: -3.892, e: 0.1, Collision: True 
episode: 4539/5000, score: -3.946, e: 0.1, Collision: True 
episode: 4540/5000, score: -3.0355, e: 0.1, Collision: True 
episode: 4541/5000, score: -3.944, e: 0.1, Collision: True 
episode: 4542/5000, score: 1.0775, e: 0.1, Collision: False 
episode: 4543/5000, score: -3.683, e: 0.1, Collision: True 
episode: 4544/5000, score: -3.926, e: 0.1, Collision: True 
episode: 4545/5000, score: 0.815, e: 0.1, Collision: True 
episode: 4546/5000, score: -3.7445, e: 0.1, Collision: True 
episode: 4547/5000, score: -3.767, e: 0.1, Collision: True 
episode: 4548/5000, score: -3.5725, e: 0.1, Collision: True 
episode: 4549/5000, score: -3.972, e: 0.1, Collision: True 
episode: 4550/5000, score: -3.974, e: 0.1, Collision: True 
episode: 4551/5000, score: -3.8, e: 0.1, Collision: True 
episode: 4552/5000, score: -3.946, e: 0

episode: 4674/5000, score: -3.853, e: 0.1, Collision: True 
episode: 4675/5000, score: -3.701, e: 0.1, Collision: True 
episode: 4676/5000, score: -3.94, e: 0.1, Collision: True 
episode: 4677/5000, score: -2.566, e: 0.1, Collision: True 
episode: 4678/5000, score: -3.9125, e: 0.1, Collision: True 
episode: 4679/5000, score: -3.52, e: 0.1, Collision: True 
episode: 4680/5000, score: -3.523, e: 0.1, Collision: True 
episode: 4681/5000, score: -3.656, e: 0.1, Collision: True 
episode: 4682/5000, score: -3.94, e: 0.1, Collision: True 
episode: 4683/5000, score: -3.73, e: 0.1, Collision: True 
episode: 4684/5000, score: -3.652, e: 0.1, Collision: True 
episode: 4685/5000, score: -3.791, e: 0.1, Collision: True 
episode: 4686/5000, score: -3.97, e: 0.1, Collision: True 
episode: 4687/5000, score: -3.911, e: 0.1, Collision: True 
episode: 4688/5000, score: -3.37, e: 0.1, Collision: True 
episode: 4689/5000, score: -3.941, e: 0.1, Collision: True 
episode: 4690/5000, score: -3.972, e: 0.1, Co

episode: 4811/5000, score: -3.946, e: 0.1, Collision: True 
episode: 4812/5000, score: -3.735, e: 0.1, Collision: True 
episode: 4813/5000, score: -3.944, e: 0.1, Collision: True 
episode: 4814/5000, score: -3.943, e: 0.1, Collision: True 
episode: 4815/5000, score: -3.94, e: 0.1, Collision: True 
episode: 4816/5000, score: -3.883, e: 0.1, Collision: True 
episode: 4817/5000, score: -3.762, e: 0.1, Collision: True 
episode: 4818/5000, score: -3.82, e: 0.1, Collision: True 
episode: 4819/5000, score: -3.793, e: 0.1, Collision: True 
episode: 4820/5000, score: -3.912, e: 0.1, Collision: True 
episode: 4821/5000, score: -3.91, e: 0.1, Collision: True 
episode: 4822/5000, score: -3.971, e: 0.1, Collision: True 
episode: 4823/5000, score: -0.701, e: 0.1, Collision: True 
episode: 4824/5000, score: -0.172, e: 0.1, Collision: True 
episode: 4825/5000, score: -3.85, e: 0.1, Collision: True 
episode: 4826/5000, score: -3.58, e: 0.1, Collision: True 
episode: 4827/5000, score: -3.881, e: 0.1, Co

episode: 4949/5000, score: -3.882, e: 0.1, Collision: True 
episode: 4950/5000, score: -3.641, e: 0.1, Collision: True 
episode: 4951/5000, score: -3.91, e: 0.1, Collision: True 
episode: 4952/5000, score: -3.608, e: 0.1, Collision: True 
episode: 4953/5000, score: -3.2015, e: 0.1, Collision: True 
episode: 4954/5000, score: -3.832, e: 0.1, Collision: True 
episode: 4955/5000, score: -3.851, e: 0.1, Collision: True 
episode: 4956/5000, score: -3.95, e: 0.1, Collision: True 
episode: 4957/5000, score: -3.919, e: 0.1, Collision: True 
episode: 4958/5000, score: -3.974, e: 0.1, Collision: True 
episode: 4959/5000, score: -3.904, e: 0.1, Collision: True 
episode: 4960/5000, score: -3.668, e: 0.1, Collision: True 
episode: 4961/5000, score: -3.975, e: 0.1, Collision: True 
episode: 4962/5000, score: -3.974, e: 0.1, Collision: True 
episode: 4963/5000, score: -3.972, e: 0.1, Collision: True 
episode: 4964/5000, score: -3.581, e: 0.1, Collision: True 
episode: 4965/5000, score: -0.6495, e: 0.

5.0
